# Hands-On Assignment 3

In this assignment, you will learn the basics of data cleaning and munging (manipulation).
Data is the heart and soul of a data scientist and machine learner.
Therefore, it is important for you to get comfortable taking data from its natural, messy state
to a state where it is ready for machine learning algorithms.

Since data is at the core of all we do, we learn to love data and treat it well.
However, not everyone is a data scientist, a computer scientist, or even a programmer.
Others that you work with may carefully collect very important data,
but handling and storing the data may not be their expertise.
So whenever working with data prepared by others,
we must always treat their data with care and remember that even data presented in suboptimal ways can be of the utmost importance.
Also remember that even as we learn better data handling practices,
there will still be those with data handling skill and methodologies that are leagues better than our own.
(It's database people, they are usually the best data people. (Written by Eriq, a database person.))

The objective of this assignment is for you to learn about:
 - Finding errors and inconsistencies in data.
 - Extracting key pieces of information from a data column.
 - Assigning types to data columns.
 - Understanding and finding outliers in your data.
 - Replacing missing pieces of data.
 - Encoding data.
 - Joining data.

## Data: CIA World Factbook - 2022

For this assignment, we will be using data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/).
This is a collection of real data about different countries and regions in the world compiled every year by the CIA.
Specifically, we will be using the world factbook data from [this repository](https://github.com/factbook/factbook.json/) from 2022.

The data can be found in your assignment repository in the `cia_world_factbook_2022.json` file.

Let's take a look at the data as we have done in previous assignments:

In [ ]:
import json

import pandas

# Our data comes in JSON, so first parse the JSON data.
with open('cia_world_factbook_2022.json', 'r') as file:
    data = json.load(file)

# Now load the JSON data into a Pandas dataframe.
world_data = pandas.DataFrame.from_dict(data, orient = 'index')

# Sort the data by country name so it looks nice.
world_data.sort_index(axis = 0, inplace = True)

# Move the country name out of the index and into an actual column.
world_data.insert(0, 'Country', world_data.index)
world_data.reset_index(drop = True, inplace = True)

world_data

Let's also take a look at the column information:

In [ ]:
world_data.info()

Now the numerical values:

In [ ]:
world_data.describe()

Note that when we tried to get information about the numerical columns like in a previous assignment,
we didn't get nearly as much information as we did before.
If we look back to `world_data.info()`,
we can see this is because Pandas doesn't actually know which of our columns are numbers
(the dtype ("data type") for each column is just `object`).
If we look at some of the values (like "38,346,720 (2022 est.)"),
we can see why Pandas would be confused about how to treat that data.
Pandas will always try and choose a data type that can be applied to every value in the column.
So if a column has a million ints and one float, then the column type will be float
(since all ints are floats, but not all floats are ints).

Let's see if we can cleanup this data!

## Part 1: Exploring Columns

The first thing to do with a new dataset is explore the columns.
 - What data is each column suppose to contain?
 - What data type should be used to represent each column?
 - Which columns may be useful?

Not all columns in each dataset are going to be useful.
We can start our work by eliminating columns that have a low chance of being needed.

<h3 style="color: darkorange";>★ Task 1.A</h3>

Complete the following function that takes in a frame and drops the columns in the frame that have greater than `sparsity_threshold` percent of values empty.
Assume any value that [Pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html) returns true for is "empty".
Return the passed-in frame.

In [ ]:
def drop_sparse_columns(frame, sparsity_threshold):
    return NotImplemented

print("Dropping any column with at least 50% empty values:")
drop_sparse_columns(world_data, 0.50)

Now that we have fewer columns to worry about,
we can take a closer look at our data.

It looks like this data was originally meant for people and not machines.
There are several things that make it easier for people to read, but harder for us machine learning folks:
 - Commas in numeric values: '38,346,720' instead of '38346720'.
 - Units after numbers: '53.65 years', '37.3%'.
 - Notes about the data interweaved with the data itself: '37.6% (2017 est.)' instead of just '37.6%' (or better: '0.376').

Externally displaying data with these traits can really help analysts and non-data scientists,
but can really get in the way of our data analysis work.
When possible, we want to store the data cleanly and translate it to a more human readable form (often called a *report*) for readers, analysts, managers, etc..
This also makes it easier to transform our data in different ways
(like using a dot instead of a comma as a decimal separator in some countries: '1,000' vs '1.000').

In our data, it looks like many columns just contain a single main number with some potential context information around it:

In [ ]:
world_data['Population'][0]

There are also more complex cases that contain multiple relevant numbers:

In [ ]:
world_data['Population'][1]

We will want to clean up these cells so that they contain just the core piece of information and not all that extra information.

### An Aside on Regular Expressions

[Regular expressions](https://en.wikipedia.org/wiki/Regular_expression) (also called "regexp" or "regex") are patterns that let you find matching text.
Think of them like mathematical expressions for text (an equation can define a line (collection of points), and a regex can define a set of strings).
Covering regular expressions is outside the scope of this course,
and they are **not necessary for any of the assignments in this course**.
However, they can be very useful in many tasks (especially in data science).
And once you get comfortable using them, you start to see how they can be used in almost all of your everyday coding.

<center><img src="xkcd-regular-expressions.png"/></center>
<center style='font-size: small'>Comic courtesy of <a href='https://xkcd.com/208'>xkcd</a></center>

We encourage you to learn about regular expressions and become comfortable using them in your everyday coding.
Here are some resources to help you:
 - [Text Tutorial](https://www.sitepoint.com/learn-regex/)
 - [Video Tutorial](https://www.youtube.com/watch?v=sa-TUpSx1JA)
 - [Cheat Sheet](https://cheatography.com/davechild/cheat-sheets/regular-expressions/)
 - [Regex Playground](https://regex101.com/) (Interactivley create, test, and visualize regular expressions.)

<h3 style="color: darkorange";>★ Task 1.B</h3>

Complete the following function that takes in a frame and a list of columns to ignore.
For all columns that are not ignored,
extract a number from the cell's text and replace the cell's contents with that correctly typed number (either a `float` or `int`).
The number should be converted to an `int` if the text does not have a decimal point (and is not a percentage).
If the cell contains no number, replace it with a `numpy.nan`.
Convert percentages to normalized floats, e.g., `str('37.6% (2017 est.)')` turns into `float(0.376)`.
Return the passed-in frame.

For cells with ambiguous data/numbers,
it is your job to do your best to honor what that data's author likely intended.

**Warning:
This is a hard task with no exact answer (as with most tasks in machine learning).
Look through the data to find as many edge cases as you can, turn those into test cases, and see how many you can solve.
Start with a simple solution and work up from there.**

In [ ]:
def extract_numbers(frame, ignore_columns = []):
    return NotImplemented

print("Cleaning up the numeric values:")
extract_numbers(world_data, ['Country', 'Export commodities'])

As a consequence of what we just did, we also standardized all the NaN values in those columns.
Before we could see empty values represented in several different ways:
 - 'NaN'
 - 'NA'
 - 'total: NA'
 - '' (empty)

Now, since any cell that didn't contain a number was replaced with `numpy.nan`,
all numeric cells with missing/empty values are consistent.

Now that we have cleaned up our numeric columns,
let's officially tell Pandas the data types for our columns.
Because even though most of our columns now only contains numbers (and `numpy.nan`),
Pandas still needs to be told what data type each column uses.

In [ ]:
world_data.describe()

<h3 style="color: darkorange";>★ Task 1.C</h3>

Complete the following function that takes in a frame and assigns the closest matching type to each column.
You can choose from the following three types: `int`, `float`, and `str`.
(Note that columns containing only integers should have the `int` type, not the `float` type.)
Ignore any `numpy.nan` values when making decisions.
Return the passed-in frame.

In [ ]:
def guess_types(frame):
    return NotImplemented

print("Data description after types are assigned:")
guess_types(world_data)
world_data.describe()

Now we can see more complete statistics for our numerical columns.
Our data is so much cleaner than when we started,
but it still has some issues lurking in it and is not ready for our machine learning algorithms.

## Part 2: Data Cleaning and Smoothing

Now that our data is cleaner, we can look closer at our numbers.
It is always important to take time to go over your data.
To help understand our data, we will use both aggregate statistics and manual inspection.

### Data Outliers

Now, you may notice some inconsistencies (depending on how well you did in previous parts)
in your data statistics (`describe()`).
For example, you may notice a value for a percentage column above 1.0 or a population in the single digits.
Looking at these aggregate statistics can help us recognize that there are issues in our data.
However, we still need to isolate the specific entries with issues and fix them.

To start, we want to look for [outliers](https://en.wikipedia.org/wiki/Outlier) in our data.
An outlier is just a point that lies "outside" the expected range for our data.
The exact definition for what is considered "outside" our expected range depends on the dataset and situation.

<h3 style="color: darkorange";>★ Task 2.A</h3>

Complete the function below which takes in a frame, a number of standard deviations, and the name of a label column.
This function will search all numeric columns for outliers.
An outlier will be any value that is more that the specified number of standard deviations away from the column's mean.
Any NaNs should be ignored.

The return value should be a dict where the keys are the names of columns that contain an outlier
and the value is a list of tuples with the value of the label column for that row and the outlier value.
Columns without outliers (and non-numeric columns) should be omitted from the returned result.

For example, a valid return from `find_outliers(world_data, 4.0, 'Country')` could be:
```
{
    'Population': [
        ('China', 1410539758.0),
        ('India', 1389637446.0)
    ]
}
```

Use this function as a chance to catch bugs you may have in your number parsing function.

In [ ]:
def find_outliers(frame, deviations, label_column_name):
    return NotImplemented

print("Outliers found in our data:")
find_outliers(world_data, 4.0, 'Country')

Once we have found outliers in our data, we have the hard decision of what to do with the outliers:
 - Are the outliers errors in the original data that we should discuss with the original data authors?
 - Are they a symptom of incorrect data parsing earlier in our pipeline?
 - Should they be replaced with a more typical value?
 - Should they be removed entirely?
 - Are they real and should just be kept in the data without modification.

In our case, we will keep our outliers intact.
However, use this as an opportunity to look for bugs in you number parsing code from Part 1.
Incorrectly parsed numbers can easily show up as outliers.

### Data Imputation

Besides outliers, another form of anomalous data we may see are *missing values*.
Missing values may be represented in many different ways depending on the dataset:
numpy.nan, -1, null, None, "MISSING", etc.

Sometimes what to do with missing data is obvious given the dataset.
For example, if a new country was founded in 2022 then the value in its "Unemployment rate 2019" should be missing.
Neither a 0.0 nor 1.0 would make sense, the value just does not exist.

However in more complex situations, deciding what to do with missing data can be very difficult.
One potential way to deal with missing data is through [data imputation](https://en.wikipedia.org/wiki/Imputation_(statistics)).
"Imputing" data is when we replace missing values with some derived value.
We are essentially "filling in the gaps" in our data with values that make sense.
There are many methods of choosing which values to impute, some of which are:
 - Using the **mean** value of the column (using non-empty values).
 - Using other values to create a machine learning model (often a regressor) to **predict** what the value should be.
 - Making a **random** guess (which can sometimes work better than you may expect).
 - Using knowledge of the domain to build a probabilistic model and then using that model to make a **educated random** guess.

There is no fixed rule or theorem that will tell you what the best data imputation method is for each dataset or column.
It will usually come down to the expertise of a data scientists (that's you!)
to bridge the gap between the domain data and statistics and figure our the best way to impute missing data (or whether to leave it be).
Always be careful when imputing values,
since it may bias your data.

In our data, the most sparsely populated columns (and candidates for imputation)
are "Literacy" and the three "Unemployment" columns.

The "Literacy" column could be a very interesting column to impute,
because there are many [studies on different predictors of literacy in adults](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0243763).
We may be able to use the other columns in our dataset (like "Electricity access", "Internet access", "Improved Sanitation", "Improved Water")
to predict literacy rate.
Unfortunately, building regressors is outside the scope of this assignment.

Still, the "Unemployment" columns also provides an interesting data imputation opportunity for us.
There are three columns that represent employment for three years ("Unemployment rate 2021", "Unemployment rate 2020", "Unemployment rate 2019"),
and each country may have values for any number of those columns.
To smooth out the data and impute missing values,
we can combine these columns into one ("Recent Unemployment") and take the **mean** of all present values.

<h3 style="color: darkorange";>★ Task 2.B</h3>

Complete the function below that takes a frame, a list of column names, and a new column name.
Modify the frame to merge the provided columns into a single column with the provided name.
To merge columns, take the mean of any present values.
If no values are present, use `numpy.nan`.
Return the passed-in frame.

In [ ]:
def merge_columns(frame, merge_column_names, new_column_name):
    return NotImplemented

print("Data with merged unemployment data:")
columns = ['Unemployment rate 2019', 'Unemployment rate 2020', 'Unemployment rate 2021']
merge_columns(world_data, columns, 'Recent Unemployment')

## Part 3: Encoding Data

Let's finally deal with that pesky "Export commodities" column!
It's not a number, but represents a list of items.
It is common to deal with data that is not numeric (arbitrary strings, strings representing single items, strings representing list/sets of items, etc).
To deal with these non-numeric columns, we need to encode them.
That is, we need to convert them from a non-numeric form to a numeric one.

In computer science, there are many encodings and encoding methods.
An encoding that you are probably already familiar with is [ASCII](https://en.wikipedia.org/wiki/ASCII),
which is a way to represent Latin letters as numbers.
Encodings are just agreed upon ways to represent non-numeric values as numbers.

In machine learning and data science, probably the most popular encoding method is [the One-Hot encoding](https://en.wikipedia.org/wiki/One-hot).
In One-Hot encoding, we take all the possible values an item can take and assign each possible value an index.
Presence of that value results in a 1, which absence results in a 0.
Essentially, we are turning a list of items into a series of binary columns which ask "Do you have this value?".

For example, consider the following tables.
In the first one, a student's major is kept as a string (or a list of strings).
To encode it, a column is created for each possible major and a 1 is present if that student has that major
(otherwise a 0 is used).

<center><img src="one-hot.png"/></center>

<h3 style="color: darkorange";>★ Task 3.A</h3>

Complete the function below that takes in a frame and a column name.
The function should modify the frame to add multiple columns represented a one-hot encoding of the specified column.
Assume that the given column contains a comma-separated list of values.
Each value should be made lowercase and have additional whitespace removed from the beginning and end.
The new column names should be named: "`<old column name>: <value>`.
(Like in the students example above.)
Return **a new frame** with the columns of the passed-in frame (minus the specified column) and all the new one-hot columns.

Note that the above semantics will not work perfectly for our world data,
but it will provide a good starting point.

In [ ]:
def one_hot(frame, column_name):
    return NotImplemented

In [ ]:
# Before calling your function, we are going to clean up the data a little bit.
# You are not required to learn what we are doing below (regular expressions),
# but they are very useful and will always be useful to you.
world_data['Export commodities'].replace(r'\s*\(\d{4}(?:\s*est\.)?\)\s*', '',
                                         regex = True, inplace = True)
world_data['Export commodities'].replace(r'\s*\(\d{2}%\)\s*', '',
                                         regex = True, inplace = True)
world_data['Export commodities'].replace(r'\s*\d{2}%\s*', '',
                                         regex = True, inplace = True)

print("One-Hot encoded exports:")
world_data = one_hot(world_data, 'Export commodities')
world_data

## Part 4: Joining Data

Data is not always from a single frame/table or source.
You will often need to merge and cross-reference data from other sources and in different frames.
When we talk about merging data from multiple frames or tables,
we usually use terminology that comes from databases and relational algebra.
So we say "table" to refer to the abstract concept of tabular data (data with rows and columns) like a Pandas.DataFrame.
Then, a [join](https://en.wikipedia.org/wiki/Join_(SQL)) is an operation that combines data from the columns of two different tables.
(Note that the same table can actually be used multiple times (called a "self-join"),
and joins can be chained together to include as many tables as desired.)

In this section, we will briefly show the main different types of joins.
The image below shows a graphical representation of the primary join types.
When discussing joins, we will refer to the table on the left side of the join operator as the LHS (left-hand side)
and the table on the right side of the join operator as the RHS (right-hand side).
In the diagram below, $ A $ is always the LHS and $ B $ is always the RHS.

Joins will typically specify some condition that must be meant for two rows to be merged,
referred to as a "join condition".
The most common join condition is equality for a set of columns.
When a collection of columns are checked for equality in a join,
we say we join "on" those columns.
For example, we may say that we join $ A $ and $ B $ on the "ID" column.

<center><img src="joins.png"/></center>

### Cross Join / Cartesian Product / Cross Product

Cross joins (also known an "Cartesian products" or "cross products")
are when every row in the LHS is paired with every row in the RHS.
Therefore, the resulting table will have $ |A| * |B| $ number of rows.

Cross joins can be though of as the basis for all other joins,
since all possible rows are enumerated.

### Inner Join

Inner joins take only the rows from the LHS and RHS where the join conditions are met.
We call an inner join where all shared columns (columns with the same name) between the LHS and RHS are checked for equality a *natural join*.

### Full Outer Join

Full outer joins (also known as "outer joins") ensure that every row from each table is represented.
First, rows where the join condition are met are included.
Then, rows from each table are included where the join condition are not met are included.
For these non-matching rows, any columns that come from the other table are filled in with null or empty values
(depending on the engine that performs the join).

### Left Outer Join

Left outer joins are like full outer joins,
except only the non-matching rows from the LHS are included.

### Right Outer Join

Left outer joins are like full outer joins,
except only the non-matching rows from the RHS are included.

Now that you are an expert in join types, let's get some new data to join!
To augment our data, we can get more information from the [World Health Organization](https://www.who.int/) (WHO).
To tie in with our previous HO, we can get some 2022 statistics for each country on their Covid-19 status.

In [ ]:
with open('who_covid_data_2022.json', 'r') as file:
    data = json.load(file)

who_data = pandas.DataFrame.from_dict(data, orient = 'index')
who_data.sort_index(axis = 0, inplace = True)
who_data.head()

To perform joins in Pandas, a useful method is [DataFrame.join()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html).
Of course, Pandas always has multiple ways to do anything, but the `DataFrame.join()` method is a good start.

<h3 style="color: darkorange";>★ Task 4.A</h3>

Complete the function below that takes in two frames and a list of column names.
This function should return a new frame that contains the left outer join between these frames on the specified columns from the LHS.

*Hint: Make sure to read the documentation on DataFrame.join() if you are going to use it.*

In [ ]:
def left_join(lhs_frame, rhs_frame, column_names):
    return NotImplemented

print("World data that includes Covid-19 stats:")
covid_world_data = left_join(world_data, who_data, ['Country'])
covid_world_data

Now we can see our country data along with Covid-19 data for each country!

In [ ]:
covid_world_data.describe()

Note that we will likely have rows from our world data that did not match up with the WHO data.
To see these, we can just look for empty values in WHO columns (since a left join would put empty values there).

In [ ]:
covid_world_data[covid_world_data['Total_Vaccinations'].isnull()]

Looking at these results, we can understand why the WHO may not have data on some of them.
Some are not technically countries (like "Antarctica"),
some have ambiguous national status (like "Taiwan"),
and some are just using different but valid names (like "United States" vs "United States of America").

As a data scientist, think about ways that you could more efficiently join together these datasets.

Congratulations!
You now have a grasp on the basics of data cleaning and munging (the soul of a data scientist).